In [0]:
pip install neattext > /dev/null

Python interpreter will be restarted.
Python interpreter will be restarted.

In [0]:
import mlflow
import mlflow.sklearn
import pandas as pd
import matplotlib.pyplot as plt

from numpy import savetxt

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import neattext.functions as nfx

In [0]:
#df = pd.read_csv("/dbfs/FileStore/ML_DataAnalytics/test.csv")

In [0]:
# mlflow.sklearn.autolog()
# with mlflow.start_run():
#     y = df[['Bluetooth', 'VC-Settings', 'Volume']]
#     corpus = df['review'].apply(nfx.remove_stopwords)
#     tfidf = TfidfVectorizer()
#     Xfeatures = tfidf.fit_transform(corpus).toarray()
#     X_train, X_test, y_train, y_test = train_test_split(Xfeatures, y, test_size=0.2, random_state=42)
#     clf = RandomForestClassifier(n_estimators=100)
#     clf.fit(X_train, y_train)
#     fr_prediction = clf.predict(X_test)
#     print(accuracy_score(y_test, fr_prediction))

In [0]:
model_name = "Text_Classification"
model = mlflow.pyfunc.load_model(f"models:/{model_name}/staging")

In [0]:
#df2 = pd.read_csv("/dbfs/FileStore/ML_DataAnalytics/Prediction_data.csv")
reviews_df = spark.table("appfigures.reviews").selectExpr("author", "review", "stars", "version", "date", "product", "product_id", "product_name", "predicted_langs as languages").filter("languages[0] = 'en' and stars < 3")
reviews_pdf = reviews_df.toPandas()

In [0]:
import joblib
tfidf_model = joblib.load('/dbfs/FileStore/ML_DataAnalytics/RF_tfidf_vectorizer.pkl')
vec_example = tfidf_model.transform(reviews_pdf['review'])
pre = model.predict(vec_example.toarray())

/local_disk0/.ephemeral_nfs/envs/pythonEnv-7375ba6c-e97d-4383-bc30-bfad43af65d8/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
 warnings.warn(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-7375ba6c-e97d-4383-bc30-bfad43af65d8/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
 warnings.warn(

In [0]:
reviews_pdf["Bluetooth"] = pre[:, 0]
reviews_pdf["VC_Settings"] = pre[:, 1]
reviews_pdf["Volume"] = pre[:, 2]

In [0]:
# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# Create a Spark DataFrame from a pandas DataFrame using Arrow
review_classified_df = spark.createDataFrame(reviews_pdf)

spark.sql("drop table if exists lake.apf_reviews_classified")

review_classified_df.write\
         .option("path", "/lake/apf/reviews_classified")\
         .option("mergeSchema", "true")\
         .mode("overwrite")\
         .saveAsTable("lake.apf_reviews_classified")

In [0]:
## optimize shuffle read

spark.sql("optimize lake.apf_reviews_classified")

Out[9]: DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint>]